## EZKL Jupyter Notebook Demo 

Here we demonstrate how to use the EZKL package to run a private network on public data to produce a public output.


In [13]:
# check if notebook is in colab
try:
    # install ezkl
    import google.colab
    import subprocess
    import sys
    subprocess.check_call([sys.executable, "-m", "pip", "install", "ezkl"])
    subprocess.check_call([sys.executable, "-m", "pip", "install", "onnx"])

# rely on local installation of ezkl if the notebook is not in colab
except:
    pass


# here we create and (potentially train a model)

# make sure you have the dependencies required here already installed
from torch import nn
import ezkl
import os
import json
import torch


# Defines the model
# we got convs, we got relu, we got linear layers
# What else could one want ????

class MyModel(nn.Module):
    def __init__(self):
        super(MyModel, self).__init__()

        self.conv1 = nn.Conv2d(in_channels=1, out_channels=2, kernel_size=5, stride=2)
        self.conv2 = nn.Conv2d(in_channels=2, out_channels=3, kernel_size=5, stride=2)

        self.relu = nn.ReLU()

        self.d1 = nn.Linear(48, 48)
        self.d2 = nn.Linear(48, 10)

    def forward(self, x):
        # 32x1x28x28 => 32x32x26x26
        x = self.conv1(x)
        x = self.relu(x)
        x = self.conv2(x)
        x = self.relu(x)

        # flatten => 32 x (32*26*26)
        x = x.flatten(start_dim = 1)

        # 32 x (32*26*26) => 32x128
        x = self.d1(x)
        x = self.relu(x)

        # logits => 32x10
        logits = self.d2(x)

        # reduce to single value
        logits = torch.mean(logits, dim=1)

        return logits


circuit = MyModel()

# Set all model weights to 0
for param in circuit.parameters():
    param.data.fill_(0)

# Train the model as you like here (skipped for brevity)




In [14]:
model_path = os.path.join('network.onnx')
compiled_model_path = os.path.join('network.compiled')
pk_path = os.path.join('test.pk')
vk_path = os.path.join('test.vk')
settings_path = os.path.join('settings.json')

witness_path = os.path.join('witness.json')
data_path = os.path.join('input.json')

In [15]:

shape = [1, 28, 28]
# After training, export to onnx (network.onnx) and create a data file (input.json)
# x = 0.1*torch.rand(1,*shape, requires_grad=True)
x = torch.ones(1,*shape, requires_grad=True) * 3
print(x.shape)

# Flips the neural net into inference mode
circuit.eval()

out = circuit(x)
print(out)

    # Export the model
torch.onnx.export(circuit,               # model being run
                      x,                   # model input (or a tuple for multiple inputs)
                      model_path,            # where to save the model (can be a file or file-like object)
                      export_params=True,        # store the trained parameter weights inside the model file
                      opset_version=10,          # the ONNX version to export the model to
                      do_constant_folding=True,  # whether to execute constant folding for optimization
                      input_names = ['input'],   # the model's input names
                      output_names = ['output'], # the model's output names
                      dynamic_axes={'input' : {0 : 'batch_size'},    # variable length axes
                                    'output' : {0 : 'batch_size'}})

data_array = ((x).detach().numpy()).reshape([-1]).tolist()

data = dict(input_data = [data_array])

    # Serialize data into file:
json.dump( data, open(data_path, 'w' ))


torch.Size([1, 1, 28, 28])
tensor([0.], grad_fn=<MeanBackward1>)


In [16]:
py_run_args = ezkl.PyRunArgs()
py_run_args.input_visibility = "public"
py_run_args.output_visibility = "hashed/public"
py_run_args.param_visibility = "private" # private by default

res = ezkl.gen_settings(model_path, settings_path, py_run_args=py_run_args)
assert res == True


In [17]:
cal_path = os.path.join("calibration.json")

data_array = (torch.rand(20, *shape, requires_grad=True).detach().numpy()).reshape([-1]).tolist()

data = dict(input_data = [data_array])

# Serialize data into file:
json.dump(data, open(cal_path, 'w'))


await ezkl.calibrate_settings(cal_path, model_path, settings_path, "resources")



 <------------- Numerical Fidelity Report (input_scale: 13, param_scale: 13, scale_input_multiplier: 10) ------------->

+------------+--------------+-----------+-----------+----------------+------------------+---------------+---------------+--------------------+--------------------+------------------------+
| mean_error | median_error | max_error | min_error | mean_abs_error | median_abs_error | max_abs_error | min_abs_error | mean_squared_error | mean_percent_error | mean_abs_percent_error |
+------------+--------------+-----------+-----------+----------------+------------------+---------------+---------------+--------------------+--------------------+------------------------+
| 0          | 0            | 0         | 0         | 0              | 0                | 0             | 0             | 0                  | 0                  | 0                      |
+------------+--------------+-----------+-----------+----------------+------------------+---------------+---------------+

True

In [18]:
res = ezkl.compile_circuit(model_path, compiled_model_path, settings_path)
assert res == True

In [19]:
# srs path
res = await ezkl.get_srs( settings_path)

In [20]:
# now generate the witness file 

res = await ezkl.gen_witness(data_path, compiled_model_path, witness_path)
assert os.path.isfile(witness_path)

In [21]:

# HERE WE SETUP THE CIRCUIT PARAMS
# WE GOT KEYS
# WE GOT CIRCUIT PARAMETERS
# EVERYTHING ANYONE HAS EVER NEEDED FOR ZK



res = ezkl.setup(
        compiled_model_path,
        vk_path,
        pk_path,
        
    )

assert res == True
assert os.path.isfile(vk_path)
assert os.path.isfile(pk_path)
assert os.path.isfile(settings_path)

In [22]:
# GENERATE A PROOF


proof_path = os.path.join('test.pf')

res = ezkl.prove(
        witness_path,
        compiled_model_path,
        pk_path,
        proof_path,
        
        "single",
    )

print(res)
assert os.path.isfile(proof_path)

{'instances': [['...']], 'proof': '0x1940df62ed86e01265e6bc0e1dc913d7111d494b6cea3b956c84f45338d08cbb2580e6f0f293c829b7dbf7945fd1570b776a225111d0a37956c03c56b7c269781ffc12cdf3c56c086f0a57760f09105b86d731c654180e468af3ac2a4e512ac42036f3a4b55302f55c608ddaaaf9b7737a2fd6291454c4850a18aed58eb536fb006680225f5f0aae518011e962414c852de3ccfb0118828049a6a79e8e01be121f3d857947f1f797753256d2e9e7f814ecfa4b4433b342186190ea694b39ec1a0fecf64681f1a65c21e0d349bc566a2bb0107be6d9324f3d048645c014e8d7dd0da008e7ff0610d81eb9c19e082e1cdd3dcff41203101a1726ef50265256e4d71ab2a7b352366ec739a9d367cf44d31d9db021b492ca688e9645fc3b61a91f672aab67e208a5e52320524d2b91f8825c69b2111dce19fc57df3443be480a52382295444f16dd670403d45b9a68994d5ae6e3e3640e504688901ef221d351dd502d4d261b44922c9f8cb51adff6a44036b2665cdef883f0e676c736b4118a51120ba7f28c3a5fc0d992ae67f13b721b54d27d61236b696dc7d3c90de4bfb4fded1954b5693b31aa82265261915979050869b4d151958def14a7a16a69940426542b9c3eb3bd2bd6c406568c843ef4cd137c496d790247c27d0319d733b819708c20c

In [23]:
# VERIFY IT

res = ezkl.verify(
        proof_path,
        settings_path,
        vk_path,
        
    )

assert res == True
print("verified")

verified
